In [2]:
import cv2, os, pyautogui, time
import matplotlib.pyplot as plt
from src.tools import *

width, height= pyautogui.size()

In [6]:
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 320)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 240)
try:
    while(True): 
        ret, frame = cap.read()
        preds = landmarks_detector.get_landmarks(frame)
        try:
            x_max, x_min, y_max, y_min = get_bound(preds)
            cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), color=(245, 245, 245), thickness=2) 
            box = get_box(get_bound(preds))
        except:
            pass
        
#         cv2.imshow('preview',warpBox(frame, box, target_height=32, target_width=64))
        X.append(warpBox(frame, box, target_height=32, target_width=64))
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
except:
    pass
finally:
    cv2.destroyAllWindows()
    cap.release()

In [94]:
# cv2.startWindowThread()
cv2.namedWindow("window", cv2.WND_PROP_FULLSCREEN)
cv2.setWindowProperty("window",cv2.WND_PROP_FULLSCREEN,cv2.WINDOW_FULLSCREEN)
image = np.zeros((height, width, 3), np.uint8)
image[:,:,:] = 215
x, y = np.random.rand(), np.random.rand()
cv2.circle(image, (int(x*width),int(y*height)), radius=30, color=(35,35,35), thickness=-2)
cv2.imshow("window", image)
cv2.waitKey(2000)
cv2.destroyAllWindows()

# Collect

## Method 1

In [17]:
X, Y = [], []

In [4]:
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 320)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 240)
cv2.namedWindow("window", cv2.WND_PROP_FULLSCREEN)
cv2.setWindowProperty("window",cv2.WND_PROP_FULLSCREEN,cv2.WINDOW_FULLSCREEN)
try:
    for i in range(100): 
        image = np.zeros((height, width, 3), np.uint8)
        image[:,:,:] = 215
        x, y = np.random.uniform(), np.random.uniform()
        Y.append([x,y])
        cv2.circle(image, (int(x*width),int(y*height)), radius=30, color=(35,35,35), thickness=-2)
        cv2.imshow("window", image)
        cv2.waitKey(500)
        ret, frame = cap.read()
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        preds = landmarks_detector.get_landmarks(frame)
        try:
            box = get_box(get_bound(preds))
        except:
            pass
        
#         cv2.imshow('preview',warpBox(frame, box, target_height=32, target_width=64))
        X.append(warpBox(frame, box, target_height=32, target_width=64))
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
except:
    pass
finally:
    cv2.destroyAllWindows()
    cap.release()

## Method 2

In [17]:
X, X_temp, Y = [], [], []

In [18]:
cap = cv2.VideoCapture(0)
frame_width, frame_height = 320, 240
cap.set(cv2.CAP_PROP_FRAME_WIDTH, frame_width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_height)
cv2.namedWindow("window", cv2.WND_PROP_FULLSCREEN)
cv2.setWindowProperty("window",cv2.WND_PROP_FULLSCREEN,cv2.WINDOW_FULLSCREEN)
class GetOutOfLoop(Exception):
    pass

target_height, target_width = 32, 64

x, y = 0.5, 0.5
image = np.zeros((height, width, 3), np.uint8)
path = [[0,0], [1,0], [0,1], [1,1]]
for i in range(10):
    x_new, y_new = np.random.uniform(), np.random.uniform()
    while abs(x_new - path[-1][0]) + abs(y_new-path[-1][0]) < 1.0:
        x_new, y_new = np.random.uniform(), np.random.uniform()
    path.append([x_new, y_new])
np.random.shuffle(np.array(path))
    
try:
    for x_new, y_new in path:
        T = 2000
        n = 150
        for xc, yc in np.linspace([x, y], [x_new, y_new], n):
            image[:,:,:] = 215
            cv2.circle(image, (int(xc*width),int(yc*height)), radius=15, color=(35,35,35), thickness=-2)
            cv2.imshow("window", image)
            cv2.waitKey(T//n)
            ret, frame = cap.read()
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#             preds = landmarks_detector.get_landmarks(frame)
#             try:
#                 box = get_box(get_bound(preds))
#             except:
#                 pass
#             X.append(warpBox(frame, box, target_height=target_height, target_width=target_width))
            X_temp.append(frame)
            Y.append([xc,yc])
            if cv2.waitKey(1) & 0xFF == ord('q'):
                raise GetOutOfLoop
        x, y = x_new, y_new
except Exception as e:
    print(e)
finally:
    cv2.destroyAllWindows()
    cap.release()

In [19]:
from ipywidgets import IntProgress
from IPython.display import display

target_height, target_width = 32, 64
max_count = len(X_temp)

f = IntProgress(min=0, max=max_count) # instantiate the bar
display(f) # display the bar

X = []
for frame in X_temp:
    preds = landmarks_detector.get_landmarks(frame)
    try:
        box = get_box(get_bound(preds))
    except:
        pass
    X.append(warpBox(frame, box, target_height=target_height, target_width=target_width))
    f.value += 1

IntProgress(value=0, max=2100)

In [20]:
X = np.array(X)/255.0
X = np.expand_dims(X, -1)
Y = np.array(Y[:len(X)])

print(X.shape, Y.shape)

(2100, 32, 64, 1) (2100, 2)


In [21]:
X_old = np.load("X.npy")
Y_old = np.load("Y.npy")

In [22]:
X = np.concatenate([X, X_old])
Y = np.concatenate([Y, Y_old])

In [125]:
np.save("X.npy", X)
np.save("Y.npy", Y)

In [115]:
X_old, Y_old = X, Y

# Train model

In [23]:
import tensorflow as tf
from tensorflow.keras import layers, models
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(target_height, target_width, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(2, activation='sigmoid'))

model.compile(optimizer='adam', loss="mse")
model.summary() 

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 62, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 31, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 29, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 14, 64)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 12, 64)         36928     
_________________________________________________________________
flatten (Flatten)            (None, 3072)              0         
_________________________________________

In [24]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(list(range(len(X))), train_size=0.9, shuffle=True)

/Users/jojo/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [25]:
history = model.fit(X[train_set], Y[train_set], batch_size=256, epochs=100, validation_data=(X[test_set], Y[test_set]))

Train on 5670 samples, validate on 630 samples
Epoch 1/100
5670/5670 [==============================] - 5s 893us/sample - loss: 0.0753 - val_loss: 0.0737
Epoch 2/100
5670/5670 [==============================] - 4s 741us/sample - loss: 0.0552 - val_loss: 0.0457
Epoch 3/100
5670/5670 [==============================] - 5s 794us/sample - loss: 0.0337 - val_loss: 0.0270
Epoch 4/100
5670/5670 [==============================] - 4s 772us/sample - loss: 0.0232 - val_loss: 0.0200
Epoch 5/100
5670/5670 [==============================] - 5s 797us/sample - loss: 0.0182 - val_loss: 0.0175
Epoch 6/100
5670/5670 [==============================] - 4s 792us/sample - loss: 0.0157 - val_loss: 0.0165
Epoch 7/100
5670/5670 [==============================] - 4s 779us/sample - loss: 0.0146 - val_loss: 0.0178
Epoch 8/100
5670/5670 [==============================] - 4s 790us/sample - loss: 0.0141 - val_loss: 0.0129
Epoch 9/100
5670/5670 [==============================] - 4s 788us/sample - loss: 0.0120 - val_los

5670/5670 [==============================] - 5s 797us/sample - loss: 0.0023 - val_loss: 0.0024
Epoch 77/100
5670/5670 [==============================] - 4s 792us/sample - loss: 0.0021 - val_loss: 0.0026
Epoch 78/100
5670/5670 [==============================] - 4s 769us/sample - loss: 0.0021 - val_loss: 0.0026
Epoch 79/100
5670/5670 [==============================] - 4s 774us/sample - loss: 0.0021 - val_loss: 0.0024
Epoch 80/100
5670/5670 [==============================] - 4s 773us/sample - loss: 0.0021 - val_loss: 0.0024
Epoch 81/100
5670/5670 [==============================] - 4s 771us/sample - loss: 0.0021 - val_loss: 0.0026
Epoch 82/100
5670/5670 [==============================] - 4s 777us/sample - loss: 0.0020 - val_loss: 0.0023
Epoch 83/100
5670/5670 [==============================] - 4s 770us/sample - loss: 0.0021 - val_loss: 0.0025
Epoch 84/100
5670/5670 [==============================] - 4s 777us/sample - loss: 0.0020 - val_loss: 0.0024
Epoch 85/100
5670/5670 [=================

# Test

Point test

In [90]:
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, frame_width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_height)
cv2.namedWindow("window", cv2.WND_PROP_FULLSCREEN)
cv2.setWindowProperty("window",cv2.WND_PROP_FULLSCREEN,cv2.WINDOW_FULLSCREEN)
image = np.zeros((height, width, 3), np.uint8)

try:
    while(True): 
        ret, frame = cap.read()
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        preds = landmarks_detector.get_landmarks(frame)
        try:
            box = get_box(get_bound(preds))
        except:
            pass
        img = warpBox(frame, box, target_height=target_height, target_width=target_width) / 255.0
        img = np.expand_dims(np.expand_dims(img, 0), -1)
        [x, y] = model.predict(img)[0]
        image[:,:,:] = 215
        cv2.circle(image, (int(x*width),int(y*height)), radius=30, color=(35,35,35), thickness=-2)
        cv2.imshow("window", image)        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            raise GetOutOfLoop
except Exception as e:
    print(e)
finally:
    cv2.destroyAllWindows()
    cap.release()

Mouse test

In [36]:
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, frame_width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_height)

try:
    while(True): 
        ret, frame = cap.read()
        if ret:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            preds = landmarks_detector.get_landmarks(frame)
            try:
                box = get_box(get_bound(preds))
            except:
                pass
            img = warpBox(frame, box, target_height=target_height, target_width=target_width) / 255.0
            img = np.expand_dims(np.expand_dims(img, 0), -1)
            [x, y] = model.predict(img)[0]
            pyautogui.moveTo(x * width, y * height, duration=0.15)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            raise GetOutOfLoop
except:
    pass
finally:
    cv2.destroyAllWindows()
    cap.release()